<a href="https://colab.research.google.com/github/afeld/python-public-policy/blob/main/lecture_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYU Wagner - Python Coding for Public Policy**

# Class 2: Manipulating/combining DataFrames and writing functions

# LECTURE

## **Today's goal**: Which Community Districts have the most 311 requests? Why might that be?

### What's a Community District?

- 59 local governance districts each run by an appointed Community Board
- Community boards advise on land use and zoning, participate in the city budget process, and address service delivery in their district.
- Community boards are each composed of <= 50 volunteer members appointed by the local borough president, half from nominations by the local City Council members.

[More info](https://en.wikipedia.org/wiki/Community_boards_of_New_York_City)

![Map of community districts from Wikipedia](https://upload.wikimedia.org/wikipedia/commons/4/41/New_York_City_community_districts.svg)

## Setup

In [1]:
import pandas as pd

In [2]:
# Make sure you see all the rows and columns in your dataframe. Use this sparingly.
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Read our cleaned 311 Service Requests dataset

In [3]:
df = pd.read_csv('https://nyu.box.com/shared/static/x3zfnpsva4kwcqj6amfchheszdv269lq.zip', low_memory=False)

## View the contents of the `community_board` column in our 311 data

In [4]:
set(df["Community Board"])

{'0 Unspecified',
 '01 BRONX',
 '01 BROOKLYN',
 '01 MANHATTAN',
 '01 QUEENS',
 '01 STATEN ISLAND',
 '02 BRONX',
 '02 BROOKLYN',
 '02 MANHATTAN',
 '02 QUEENS',
 '02 STATEN ISLAND',
 '03 BRONX',
 '03 BROOKLYN',
 '03 MANHATTAN',
 '03 QUEENS',
 '03 STATEN ISLAND',
 '04 BRONX',
 '04 BROOKLYN',
 '04 MANHATTAN',
 '04 QUEENS',
 '05 BRONX',
 '05 BROOKLYN',
 '05 MANHATTAN',
 '05 QUEENS',
 '06 BRONX',
 '06 BROOKLYN',
 '06 MANHATTAN',
 '06 QUEENS',
 '07 BRONX',
 '07 BROOKLYN',
 '07 MANHATTAN',
 '07 QUEENS',
 '08 BRONX',
 '08 BROOKLYN',
 '08 MANHATTAN',
 '08 QUEENS',
 '09 BRONX',
 '09 BROOKLYN',
 '09 MANHATTAN',
 '09 QUEENS',
 '10 BRONX',
 '10 BROOKLYN',
 '10 MANHATTAN',
 '10 QUEENS',
 '11 BRONX',
 '11 BROOKLYN',
 '11 MANHATTAN',
 '11 QUEENS',
 '12 BRONX',
 '12 BROOKLYN',
 '12 MANHATTAN',
 '12 QUEENS',
 '13 BROOKLYN',
 '13 QUEENS',
 '14 BROOKLYN',
 '14 QUEENS',
 '15 BROOKLYN',
 '16 BROOKLYN',
 '17 BROOKLYN',
 '18 BROOKLYN',
 '26 BRONX',
 '27 BRONX',
 '28 BRONX',
 '55 BROOKLYN',
 '56 BROOKLYN',
 '64

## Get the count of 311 requests per Community District

In [5]:
cb_counts = df.groupby('Community Board').size().reset_index(name='count_of_311_requests')
cb_counts = cb_counts.sort_values('count_of_311_requests', ascending=False)
cb_counts

,Community Board,count_of_311_requests
50,12 MANHATTAN,81403
23,05 QUEENS,71506
51,12 QUEENS,70362
2,01 BROOKLYN,68104
12,03 BROOKLYN,66360
5,01 STATEN ISLAND,65147
31,07 QUEENS,63634
21,05 BROOKLYN,61839
16,04 BRONX,61088
4,01 QUEENS,60427


## **Research Question:** What may account for the variance in count of requests per community district?

## **Hypothesis:** Population size may help explain the variance.

We can combine the counts per community district dataset with population data for each community district.

## Let's load the population dataset and check out its contents

[Data source for population by Community District](https://data.cityofnewyork.us/City-Government/New-York-City-Population-By-Community-Districts/xi7c-iiu2/data)

In [6]:
population = pd.read_csv('https://data.cityofnewyork.us/api/views/xi7c-iiu2/rows.csv')
population.head()

,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population
0,Bronx,1,"Melrose, Mott Haven, Port Morris",138557,78441,77214,82159,91497
1,Bronx,2,"Hunts Point, Longwood",99493,34399,39443,46824,52246
2,Bronx,3,"Morrisania, Crotona Park East",150636,53635,57162,68574,79762
3,Bronx,4,"Highbridge, Concourse Village",144207,114312,119962,139563,146441
4,Bronx,5,"University Hts., Fordham, Mt. Hope",121807,107995,118435,128313,128200


## In order to join the two dataframes, we need to create a common ID in each.

[`BORO CODE`](https://www1.nyc.gov/assets/planning/download/pdf/data-maps/open-data/pluto_datadictionary.pdf#page=38) (a.k.a. `BoroCode`, `borocd`, and `boro_cd`) is a commonly-used a unique ID for community districts. Let's create functions that create that unique ID in our datasets.

**BoroCD** is a 3 digit integer that captures the borough and district number. The borough is represented by the first digit. The district number is padded with zeros so it's always two digits long.

Boroughs are recoded into the following numbers:
- 1: Manhattan
- 2: Bronx
- 3: Brooklyn
- 4: Queens
- 5: Staten Island

Ex: 
- Manhattan 12 --> 112
- Brooklyn 6 --> 306


### First, let's create a `borocd` column in `cb_counts` dataframe

In [7]:
cb_counts.head()

,Community Board,count_of_311_requests
50,12 MANHATTAN,81403
23,05 QUEENS,71506
51,12 QUEENS,70362
2,01 BROOKLYN,68104
12,03 BROOKLYN,66360


Let's create a function called `recode_borocd_counts` that converts the `Community Board` value into a `borocd` value.

In [8]:
def recode_borocd_counts(row):
  if 'MANHATTAN' in row["Community Board"]:
    return '1' + row["Community Board"][0:2]
    # [0:2] provides the first 2 characters, i.e. characters at indexes 0 and 1.
    # you could also use [:2] without the zero.
  elif 'BRONX' in row["Community Board"]:
    return '2' + row["Community Board"][0:2]
  elif 'BROOKLYN' in row["Community Board"]:
    return '3' + row["Community Board"][0:2]
  elif 'QUEENS' in row["Community Board"]:
    return '4' + row["Community Board"][0:2]
  elif 'STATEN ISLAND' in row["Community Board"]:
    return '5' + row["Community Board"][0:2]
  else:
    return 'Invalid BoroCD'

In [9]:
type(recode_borocd_counts)

function

**Note:** The order of steps is important. You have to define a function before you can apply it to a dataframe.

In [10]:
cb_counts['borocd'] = cb_counts.apply(recode_borocd_counts, axis=1)

- `axis = 1` specifies that you want to apply the function across the rows instead of columns
- `cb_counts['borocd']` creates a new column in the dataframe called `borocd`
- `cb_counts.apply(function, axis)` applies the function we defined across the specified axis

In [11]:
cb_counts

,Community Board,count_of_311_requests,borocd
50,12 MANHATTAN,81403,112
23,05 QUEENS,71506,405
51,12 QUEENS,70362,412
2,01 BROOKLYN,68104,301
12,03 BROOKLYN,66360,303
5,01 STATEN ISLAND,65147,501
31,07 QUEENS,63634,407
21,05 BROOKLYN,61839,305
16,04 BRONX,61088,204
4,01 QUEENS,60427,401


Uh oh, there are some unexpected `Unspecified` values in here - how can we get around them?

Let's only recode records that don't start with "U".

In [12]:
def recode_borocd_counts(row):
  if 'MANHATTAN' in row["Community Board"] and row["Community Board"][0] != 'U':
      return '1' + row["Community Board"][:2]
  elif 'BRONX' in row["Community Board"] and row["Community Board"][0] != 'U':
      return '2' + row["Community Board"][:2]
  elif 'BROOKLYN' in row["Community Board"] and row["Community Board"][0] != 'U':
      return '3' + row["Community Board"][:2]
  elif 'QUEENS' in row["Community Board"] and row["Community Board"][0] != 'U':
      return '4' + row["Community Board"][:2]
  elif 'STATEN ISLAND' in row["Community Board"] and row["Community Board"][0] != 'U':
      return '5' + row["Community Board"][:2]
  else:
    return 'Invalid BoroCD'

cb_counts['borocd'] = cb_counts.apply(recode_borocd_counts, axis=1)
cb_counts

,Community Board,count_of_311_requests,borocd
50,12 MANHATTAN,81403,112
23,05 QUEENS,71506,405
51,12 QUEENS,70362,412
2,01 BROOKLYN,68104,301
12,03 BROOKLYN,66360,303
5,01 STATEN ISLAND,65147,501
31,07 QUEENS,63634,407
21,05 BROOKLYN,61839,305
16,04 BRONX,61088,204
4,01 QUEENS,60427,401


We can make this function easier to read by isolating the logic that applies to all the conditions. This is called "refactoring".

In [13]:
def recode_borocd_counts(row):
    board = row["Community Board"]

    if board[0] != 'U':
        num = board[0:2]
        
        if 'MANHATTAN' in board:
            return '1' + num
        elif 'BRONX' in board:
            return '2' + num
        elif 'BROOKLYN' in board:
            return '3' + num
        elif 'QUEENS' in board:
            return '4' + num
        elif 'STATEN ISLAND' in board:
            return '5' + num
    else:
        return 'Invalid BoroCD'

In [14]:
cb_counts['borocd'] = cb_counts.apply(recode_borocd_counts, axis=1)
cb_counts

,Community Board,count_of_311_requests,borocd
50,12 MANHATTAN,81403,112
23,05 QUEENS,71506,405
51,12 QUEENS,70362,412
2,01 BROOKLYN,68104,301
12,03 BROOKLYN,66360,303
5,01 STATEN ISLAND,65147,501
31,07 QUEENS,63634,407
21,05 BROOKLYN,61839,305
16,04 BRONX,61088,204
4,01 QUEENS,60427,401


### Next, let's create the `borocd` column in the population dataset

In [15]:
population.head()

,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population
0,Bronx,1,"Melrose, Mott Haven, Port Morris",138557,78441,77214,82159,91497
1,Bronx,2,"Hunts Point, Longwood",99493,34399,39443,46824,52246
2,Bronx,3,"Morrisania, Crotona Park East",150636,53635,57162,68574,79762
3,Bronx,4,"Highbridge, Concourse Village",144207,114312,119962,139563,146441
4,Bronx,5,"University Hts., Fordham, Mt. Hope",121807,107995,118435,128313,128200


Create a function `recode_borocd_pop` that combines and recodes the Borough and CD Number values to create a BoroCD unique ID.

In [16]:
def recode_borocd_pop(row):
  if row.Borough == 'Manhattan':
    return str(100 + row['CD Number'])
  elif row.Borough == 'Bronx':
    return str(200 + row['CD Number'])
  elif row.Borough == 'Brooklyn':
    return str(300 + row['CD Number'])
  elif row.Borough == 'Queens':
    return str(400 + row['CD Number'])
  elif row.Borough == 'Staten Island':
    return str(500 + row['CD Number'])
  else:
    return 'Invalid BoroCD'

In [17]:
population['borocd'] = population.apply(recode_borocd_pop, axis=1)
population

,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population,borocd
0,Bronx,1,"Melrose, Mott Haven, Port Morris",138557,78441,77214,82159,91497,201
1,Bronx,2,"Hunts Point, Longwood",99493,34399,39443,46824,52246,202
2,Bronx,3,"Morrisania, Crotona Park East",150636,53635,57162,68574,79762,203
3,Bronx,4,"Highbridge, Concourse Village",144207,114312,119962,139563,146441,204
4,Bronx,5,"University Hts., Fordham, Mt. Hope",121807,107995,118435,128313,128200,205
5,Bronx,6,"East Tremont, Belmont",114137,65016,68061,75688,83268,206
6,Bronx,7,"Bedford Park, Norwood, Fordham",113764,116827,128588,141411,139286,207
7,Bronx,8,"Riverdale, Kingsbridge, Marble Hill",103543,98275,97030,101332,101731,208
8,Bronx,9,"Soundview, Parkchester",166442,167627,155970,167859,172298,209
9,Bronx,10,"Throgs Nk., Co-op City, Pelham Bay",84948,106516,108093,115948,120392,210


## Join the population data onto the counts data after creating shared `borocd` unique ID

To join dataframes together, we will use the [pandas `.merge()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [18]:
merged_data = pd.merge(left=cb_counts, right=population, left_on='borocd', right_on='borocd')
merged_data

,Community Board,count_of_311_requests,borocd,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population
0,12 MANHATTAN,81403,112,Manhattan,12,"Washington Heights, Inwood",180561,179941,198192,208414,190020
1,05 QUEENS,71506,405,Queens,5,"Ridgewood, Glendale, Maspeth",161022,150142,149126,165911,169190
2,12 QUEENS,70362,412,Queens,12,"Jamaica, St. Albans, Hollis",206639,189383,201293,223602,225919
3,01 BROOKLYN,68104,301,Brooklyn,1,"Williamsburg, Greenpoint",179390,142942,155972,160338,173083
4,03 BROOKLYN,66360,303,Brooklyn,3,Bedford Stuyvesant,203380,133379,138696,143867,152985
5,01 STATEN ISLAND,65147,501,Staten Island,1,"Stapleton, Port Richmond",135875,138489,137806,162609,175756
6,07 QUEENS,63634,407,Queens,7,"Flushing, Bay Terrace",207589,204785,220508,242952,247354
7,05 BROOKLYN,61839,305,Brooklyn,5,"East New York, Starrett City",170791,154931,161350,173198,182896
8,04 BRONX,61088,204,Bronx,4,"Highbridge, Concourse Village",144207,114312,119962,139563,146441
9,01 QUEENS,60427,401,Queens,1,"Astoria, Long Island City",185925,185198,188549,211220,191105


## Calculate 311 requests per capita

In [19]:
# divide request count by 2010 population to get request per capita

merged_data['request_per_capita'] = merged_data.count_of_311_requests / merged_data['2010 Population']

merged_data.head()

,Community Board,count_of_311_requests,borocd,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population,request_per_capita
0,12 MANHATTAN,81403,112,Manhattan,12,"Washington Heights, Inwood",180561,179941,198192,208414,190020,0.428392
1,05 QUEENS,71506,405,Queens,5,"Ridgewood, Glendale, Maspeth",161022,150142,149126,165911,169190,0.422637
2,12 QUEENS,70362,412,Queens,12,"Jamaica, St. Albans, Hollis",206639,189383,201293,223602,225919,0.311448
3,01 BROOKLYN,68104,301,Brooklyn,1,"Williamsburg, Greenpoint",179390,142942,155972,160338,173083,0.393476
4,03 BROOKLYN,66360,303,Brooklyn,3,Bedford Stuyvesant,203380,133379,138696,143867,152985,0.433768


Let's create a simplified new dataframe that only include the columns we care about and in a better order.

In [20]:
columns = ['borocd', 'Borough', 'CD Name', '2010 Population', 'count_of_311_requests', 'request_per_capita']
cd_data = merged_data[columns]

cd_data

,borocd,Borough,CD Name,2010 Population,count_of_311_requests,request_per_capita
0,112,Manhattan,"Washington Heights, Inwood",190020,81403,0.428392
1,405,Queens,"Ridgewood, Glendale, Maspeth",169190,71506,0.422637
2,412,Queens,"Jamaica, St. Albans, Hollis",225919,70362,0.311448
3,301,Brooklyn,"Williamsburg, Greenpoint",173083,68104,0.393476
4,303,Brooklyn,Bedford Stuyvesant,152985,66360,0.433768
5,501,Staten Island,"Stapleton, Port Richmond",175756,65147,0.370667
6,407,Queens,"Flushing, Bay Terrace",247354,63634,0.257259
7,305,Brooklyn,"East New York, Starrett City",182896,61839,0.338110
8,204,Bronx,"Highbridge, Concourse Village",146441,61088,0.417151
9,401,Queens,"Astoria, Long Island City",191105,60427,0.316198


Let's check out which Community Districts have the highest complaints per capita

In [21]:
cd_data.sort_values('request_per_capita', ascending=False).head(10)

,borocd,Borough,CD Name,2010 Population,count_of_311_requests,request_per_capita
40,105,Manhattan,Midtown Business District,51673,37470,0.725137
31,308,Brooklyn,Crown Heights North,96317,43875,0.455527
29,302,Brooklyn,"Brooklyn Heights, Fort Greene",99617,44061,0.442304
21,304,Brooklyn,Bushwick,112634,49552,0.439938
4,303,Brooklyn,Bedford Stuyvesant,152985,66360,0.433768
32,309,Brooklyn,"Crown Heights South, Wingate",98429,42655,0.433358
20,110,Manhattan,Central Harlem,115723,50025,0.432282
0,112,Manhattan,"Washington Heights, Inwood",190020,81403,0.428392
1,405,Queens,"Ridgewood, Glendale, Maspeth",169190,71506,0.422637
8,204,Bronx,"Highbridge, Concourse Village",146441,61088,0.417151


While Inwood (112) had the highest number of complaints, it ranks further down on the list for requests per capita. Midtown may also be an outlier, based on it's low residential population.

## Next class we'll produce charts and maps to better visualize the differences in magnitude of the 311 requests per capita values.

# Homework 2

1. Open [the notebook](https://colab.research.google.com/github/afeld/python-public-policy/blob/main/hw_2.ipynb)
1. Save a copy in Drive